In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("..")
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs


In [3]:
from config import settings

In [4]:
where_is = WhereIs(False)

In [5]:
all_paths = where_is.get_paths_list

In [6]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0], sample_ratio=0.2)


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
customers_df = load_customers(all_paths[1], sample_ratio=0.3)

In [8]:
test_file, lnr = load_test_file(all_paths[6])

In [9]:
clean_up = CleanUp(all_paths)



In [10]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

In [11]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[5], info_df=all_paths[9],attr_df=all_paths[8])

/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)


In [12]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool

In [13]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())


I see 1 GPU devices


In [14]:
df_to_model = pd.concat(
    [dfs_dict['customers_df'].assign(is_customer=1),
     dfs_dict['azdias'].assign(is_customer=0)]
)

In [15]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [16]:
cat_features


Index(['CAMEO_DEUG_2015', 'CAMEO_DEUINTL_2015', 'CAMEO_DEU_2015',
       'OST_WEST_KZ'],
      dtype='object')

In [17]:
## APAGAR DEPOIS TESTE DE TRANSFORM ALL FLOATS TO INT


In [48]:
df_to_transform = dfs_dict['customers_df'][cat_features]

In [42]:
## Apagar depois, experimento de tudo virar int
#df_to_transform = df_to_transform.apply(pd.to_numeric, errors='coerce')

In [20]:
#b = {isinstance(item, (int,str)) for item in df_to_model[cat_features[0]].value_counts().index.tolist()}

In [ ]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

In [22]:
data_types_bool = {key: {isinstance(item, (int,str)) for item in df_to_transform[key].value_counts().index.tolist()} for key in cat_features}


In [49]:
for key, values in data_types_bool.items():
    if len(values) >1:
        print('Indo a coluna ', key)
        df_to_transform[key] = pd.to_numeric(df_to_transform[key], downcast='signed')
        df_to_transform[key] = df_to_transform[key].apply(lambda x: int(x) if not np.isnan(x) else '')
        df_to_transform[key] = np.where(df_to_transform[key] == '', np.nan, df_to_transform[key])


Indo a coluna  CAMEO_DEUG_2015
Indo a coluna  CAMEO_DEUINTL_2015


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/ipykernel_launch

In [50]:
data_types = {key: {item for item in df_to_transform[key].value_counts().index.tolist()} for key in cat_features}

In [61]:
data_types['CAMEO_DEUG_2015'][0]

TypeError: 'set' object is not subscriptable

In [38]:
X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

NameError: name 'features' is not defined

In [21]:
params_catboost = dict(num_trees=40,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)

### Mlflow data

In [24]:
tags = {'algo':'catboost', 'sample':0.2,'datasets':'population_customers', 'cols':cat_features}
run_name = 'first_cat'


In [ ]:
#mlflow.set_tracking_uri('././mlruns/')
#experiment_id = mlflow.create_experiment('CatBoostClassifier')



## Set new experiment

In [28]:
mlflow.set_experiment(experiment_name='CatBoostClassifier')


## Defining classifcator


In [23]:
catboost = CatBoostClassifier(**params_catboost)

##

In [29]:
with mlflow.start_run(tags=tags):
    mlflow.log_params(params_catboost)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    metrics_returned = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(metrics_returned)


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=10]=2.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [22]:
#Investigating error
X_train[cat_features].isna().sum().sum()

0

In [24]:
X_train[cat_features].dtypes

CAMEO_DEUG_2015       object
CAMEO_DEUINTL_2015    object
CAMEO_DEU_2015        object
OST_WEST_KZ           object
dtype: object

In [25]:
X_train[cat_features].head()

,CAMEO_DEUG_2015,CAMEO_DEUINTL_2015,CAMEO_DEU_2015,OST_WEST_KZ
30576,2.0,13.0,2B,W
532060,3,25,3D,O
147533,2,13,2B,W
507110,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
60111,2,14,2D,W


In [38]:
type(X_train[cat_features].iloc[0,0])


str

In [44]:
X_train['CAMEO_DEUG_2015'] = X_train['CAMEO_DEUG_2015'].astype('str')
X_valid['CAMEO_DEUG_2015'] = X_valid['CAMEO_DEUG_2015'].astype('str')

In [46]:
X_train['CAMEO_DEUINTL_2015'] = X_train['CAMEO_DEUINTL_2015'].astype('str')
X_valid['CAMEO_DEUINTL_2015'] = X_valid['CAMEO_DEUINTL_2015'].astype('str')

In [48]:
show_metrics_baseline(catboost, features=features, labels=labels)


Accuracy Train: 0.7634682277084924
AUC Train: 0.8651065458755732


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=10]=3.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
#Predict the testfile

